In [27]:
Output = c("/Users/alexis/Library/CloudStorage/OneDrive-UniversityofNorthCarolinaatChapelHill/CEMALB_DataAnalysisPM/Projects/P1011. Emission Mixtures/P1011.3. Analyses/P1011.3.2. Biomarker Distribution Analysis/Output")
cur_date = "060923"

library(readxl)
library(openxlsx)
library(tidyverse)
library(reshape2)
library(rlang)
library(PMCMRplus)

# reading in files
cytokine_df = data.frame(read_excel("Input/Cytokine_Data_050423.xlsx", sheet = 2))
proteomics_5_df = data.frame(read_excel("Input/Proteomics_Data_5ug_040423.xlsx", sheet = 2))
proteomics_25_df = data.frame(read_excel("Input/Proteomics_Data_25ug_040423.xlsx", sheet = 2))
mRNA_df = data.frame(read_excel("Input/Imputed_mRNA_Data_042623.xlsx"))

Warning message:
“Expecting numeric in E1226 / R1226C5: got 'NA'”
Warning message:
“Expecting numeric in E1227 / R1227C5: got 'NA'”
Warning message:
“Expecting numeric in E1228 / R1228C5: got 'NA'”
Warning message:
“Expecting numeric in E1229 / R1229C5: got 'NA'”
Warning message:
“Expecting numeric in E1230 / R1230C5: got 'NA'”
Warning message:
“Expecting numeric in E1231 / R1231C5: got 'NA'”
Warning message:
“Expecting numeric in E1232 / R1232C5: got 'NA'”
Warning message:
“Expecting numeric in E1233 / R1233C5: got 'NA'”
Warning message:
“Expecting numeric in E1234 / R1234C5: got 'NA'”
Warning message:
“Expecting numeric in E1235 / R1235C5: got 'NA'”
Warning message:
“Expecting numeric in E1236 / R1236C5: got 'NA'”
Warning message:
“Expecting numeric in E1237 / R1237C5: got 'NA'”
Warning message:
“Expecting numeric in E1238 / R1238C5: got 'NA'”
Warning message:
“Expecting numeric in E1239 / R1239C5: got 'NA'”
Warning message:
“Expecting numeric in E1240 / R1240C5: got 'NA'”
Warning me

In [28]:
# first combining the proteomics dfs and adding a col for the concentration
proteomics_25_df$Concentration = 25
proteomics_5_df$Concentration = 5

proteomics_df = rbind(proteomics_25_df, proteomics_5_df)

In [29]:
head(cytokine_df)
head(proteomics_df)
head(mRNA_df)

,Subject_No,Subject_ID,Condensate,Burn_Condition,Condensate_Conc,Cytokine,Cytokine_Conc,Cytokine_Conc_pslog2
,<dbl>,<chr>,<chr>,<chr>,<dbl>,<chr>,<dbl>,<dbl>
1,1,F_1,C,F,1,Eotaxin3,21.750958,4.507855
2,1,F_1,C,F,25,Eotaxin3,11.624224,3.658123
3,1,F_1,C,F,5,Eotaxin3,17.828714,4.234863
4,1,F_1,C,S,1,Eotaxin3,22.798534,4.572801
5,1,F_1,C,S,25,Eotaxin3,10.342567,3.503675
6,1,F_1,C,S,5,Eotaxin3,9.140652,3.342078


,Subject_No,Subject_ID,Protein_Accession,Gene_Name,Description,Condensate,Burn_Condition,Concentration,Intensity,Intensity_pslog2
,<dbl>,<chr>,<chr>,<chr>,<chr>,<chr>,<chr>,<dbl>,<dbl>,<dbl>
1,1,F_1,CO3_HUMAN,C3,Complement C3 OS=Homo sapiens OX=9606 GN=C3 PE=1 SV=2,C,F,25,1488600000,30.47131
2,2,M_2,CO3_HUMAN,C3,Complement C3 OS=Homo sapiens OX=9606 GN=C3 PE=1 SV=2,C,F,25,3806600000,31.82586
3,3,M_3,CO3_HUMAN,C3,Complement C3 OS=Homo sapiens OX=9606 GN=C3 PE=1 SV=2,C,F,25,3900000000,31.86083
4,4,F_4,CO3_HUMAN,C3,Complement C3 OS=Homo sapiens OX=9606 GN=C3 PE=1 SV=2,C,F,25,5373200000,32.32313
5,5,F_5,CO3_HUMAN,C3,Complement C3 OS=Homo sapiens OX=9606 GN=C3 PE=1 SV=2,C,F,25,2595200000,31.27320
6,6,M_6,CO3_HUMAN,C3,Complement C3 OS=Homo sapiens OX=9606 GN=C3 PE=1 SV=2,C,F,25,5780700000,32.42860


,Subject_ID,Condensate,Burn_Condition,Concentration,Time_Point,mRNA,ddCT_pslog2
,<chr>,<chr>,<chr>,<chr>,<dbl>,<chr>,<dbl>
1,M_6,PBS,PBS,NA,24,HMOX1,4.871886
2,M_6,PBS,PBS,NA,24,ALDH3A1,4.837012
3,M_6,PBS,PBS,NA,24,CXCL1,4.822372
4,M_6,PBS,PBS,NA,24,CXCR1,3.940873
5,M_6,PBS,PBS,NA,24,GCLC,4.863186
6,M_6,PBS,PBS,NA,24,GCLM,4.834127


# Research Question: Are there statistically significant differences in each biomarker between burn conditions within the condensate samples?

Testing for statistical differences by comparing an individual biomarker's value (ie. concentration, intensity or ddCT) between burn conditions (ie. control, smoldering, and flaming) within each condensate (ie. plastic and cardboard) time point (ie. 4, 24, 72).

A Friedman test is a non-paramteric alternative to an ANOVA. A Nemenyi post-hoc will be subsequently run. https://www.rdocumentation.org/packages/PMCMR/versions/4.3/topics/posthoc.friedman.nemenyi.test
https://www.statology.org/friedman-test-r/


# Friedman Test

Ideas: 

Compare across burn conditions for ANOVA, stratify within time points and condensate??

In [30]:
new = proteomics_5_df %>%
    filter(Condensate != "C", Gene_Name == "C3")
new %>%
    group_by(Burn_Condition, Subject_ID) %>%
    count()

Burn_Condition,Subject_ID,n
<chr>,<chr>,<int>
F,F_1,1
F,F_4,1
F,F_5,1
F,M_2,1
F,M_3,1
F,M_6,1
PBS,F_1,1
PBS,F_4,1
PBS,F_5,1


In [38]:
test = friedman.test(new$Intensity_pslog2, new$Burn_Condition, new$Subject_ID)
test$statistic
test$p.value

posthoc = frdAllPairsNemenyiTest(new$Intensity_pslog2, new$Burn_Condition, new$Subject_ID)
posthoc$statistic
posthoc$p.value

Friedman chi-squared 
                   1

[1] 0.6065307

,F,PBS
PBS,1.224745,NA
S,1.224745,0


,F,PBS
PBS,0.6617633,NA
S,0.6617633,1


In [ ]:
friedmans_burn_condition = function(df, concentration, value, biomarker){
    # """
    # Running Friedman's tests comparing burn conditions after filtering for biomarkers, condensate, concentration. 
    # Ultimately using this test to compare biomarker expression (control, smoldering, flaming).

    # :param: biomarker dataframe
    # :output: a dataframe containing the biomarker, comparison, concentration, FC, stat, p value, p adj

    # """
    
    # variables that will be iterated through
    unique_bioimarker = unique(df[[bioimarker]])
    conc = na.omit(unique(df[[concentration]]))
    
    values_df = data.frame()
    # iterating through each concentration
    for(i in 1:length(unique_bioimarker)){
        for(j in 1:length(conc)){

            # wilcoxon rank sum
            wilcoxon_test = wilcox.test(control_df[[value]], flaming_df[[value]], paired = TRUE)

            # calculating fold change to get directionality
            FC = log2(mean(flaming_df[[value]])/mean(control_df[[value]]))

            # contains burn condition comparison, conc, FC, stat, and p value
            values_vector = cbind(biomarker, "Smoldering vs. Flaming", conc[i], NA, FC, wilcoxon_test$statistic, 
                                  wilcoxon_test$p.value)
            values_df = rbind(values_df, values_vector)
        }

    
    # adding col names
    colnames(values_df) = c("Biomarker", "Comparison", "Concentration", "Time Point", "log2FC", "Statistic", 
                            "P Value")
    
    # calculating padj values
    values_df$`P Adj` = p.adjust(as.numeric(as.character(values_df$`P Value`)), method = "fdr")

    
    return(values_df)
}

# calling fn
smol_flaming_wilcox_cytokine = friedmans_burn_condition(cytokine_df, "Condensate_Conc", 
                                                                "Cytokine_Conc_pslog2", "Cytokine")
smol_flaming_wilcox_proteomics = friedmans_burn_condition(proteomics_df, "Concentration",
                                                                  "Intensity_pslog2", "Protein")

head(smol_flaming_wilcox_cytokine)